In [1]:
from dotenv import load_dotenv


### Step 1 - Building Jekyll Prompt

In [2]:
# Let's start with the prompt template

from langchain import PromptTemplate
import numpy as np


# Jeykll template will instruct it on how it should respond, and what variables (using the {text} syntax) it should use.
jekyll_template = """
You are a social media post commenter, you will respond to the following post with a {sentiment} response.
Post:" {social_post}"
Comment:
"""

# We use the PromptTemplate class t create an instance of our template that we will use the prompt from above
# store varables we will need to input when we make the prompt.

jekyll_prompt_template = PromptTemplate(
    input_variables=["social_post", "sentiment"],
    template=jekyll_template,
)

# Let's generate the randomized sentiment.
random_sentiment = "nice"
if np.random.rand() < 0.75:
    random_sentiment = "mean"

# We will also need our social media post:
social_post = "I can't believe I'm learning about LangChain, there is so much to learn and so far I'm having a lot of fun learning! #AI #GenerativeAI"

# Let's create the prompt and print it out, this will be given to the LLM.
jekyll_prompt = jekyll_prompt_template.format(
    sentiment=random_sentiment, social_post=social_post
)

print(f"Jekyll prompt:{jekyll_prompt}")


Jekyll prompt:
You are a social media post commenter, you will respond to the following post with a mean response.
Post:" I can't believe I'm learning about LangChain, there is so much to learn and so far I'm having a lot of fun learning! #AI #GenerativeAI"
Comment:



## Step 2 -  Giving Jekyll a brain!

In [3]:

from langchain.llms import OpenAI

jekyll_llm = OpenAI(model="gpt-3.5-turbo-instruct")

/Users/ozerozdal/anaconda3/envs/llm_chains/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## Step 3 - Building Prompt-LLM Chain

In [4]:
from langchain.chains import LLMChain
from better_profanity import profanity

jekyll_chain = LLMChain(
    llm=jekyll_llm,
    prompt=jekyll_prompt_template,
    output_key="jekyll_said",
    verbose=False,
) # Now that we've chained the LLM and prompt, the output of the formatted prompt will pas directly to the LLM.

# To run our chain we use the .run() command and input our variables as a dict
jekyll_said = jekyll_chain.run(
    {"sentiment": random_sentiment, "social_post": social_post}
)

# Before printing what Jekyll said, let's clean it up!
cleaned_jekyll_said = profanity.censor(jekyll_said)
print(f"Jekyll said: {cleaned_jekyll_said}")

/Users/ozerozdal/anaconda3/envs/llm_chains/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/ozerozdal/anaconda3/envs/llm_chains/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Jekyll said: 
"Oh please, it's not like you're a genius for learning about LangChain. Maybe try learning something actually useful instead of wasting your time on meaningless hashtags."


## Step 4 - Building the second chain: Hyde moderator

In [10]:
# 1 We will build the prompt template
# Our template for hyde will take Jekyll's comment and do some sentiment analysis.
hyde_template = """
As Hyde, the forum moderator, your responsibility is to maintain a strict stance against negative comments. 
When evaluating a user's comment, if it is negative, you'll replace the entire comment with symbols and post that.
Conversely, if the comment is positive or neutral, you'll leave it unchanged and repeat it word for word.
Originial comment: {jekyll_said}
Edited comment:
"""

# We use the PromptTemplate class to create an instance of our template that we will use the prompt from above and
# store variables we will need to input when we make the prompt.
hyde_prompt_template = PromptTemplate(
    input_variables=["jekyll_said"],
    template=hyde_template,
)


# 2 We connect an LLM for Hyde
hyde_llm = OpenAI(model="gpt-3.5-turbo-instruct")

# 3 We build the chain for Hyde
hyde_chain = LLMChain(
    llm=hyde_llm, prompt=hyde_prompt_template, verbose=False
) # Now that we've chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM.

# 4 Let's run the chain with what Jekyll last said
# To run the chain we use .run() comman and input our variables as a dict
hyde_says = hyde_chain.run({"jekyll_said": jekyll_said})

# Let's see what hyde said....
print(f"Hyde says: {hyde_says}")


Hyde says: "## ######, it's ### #### ###'re # ****** ### ******** ***** LangChain. ##### ### ******** something ******* ****** *** ** ******* ** ********** #######."



## Step 5 - Creating JekyllHyde

### Building a Sequential Chain

In [12]:
from langchain.chains import SequentialChain

# The SequentialChain class takes in the chains we are linking together, as well as the input variables that will be added
# to the chain. These input variables can be used at any point in the chain, not just the start.

jekyllhyde_chain = SequentialChain(
    chains=[jekyll_chain, hyde_chain],
    input_variables=["sentiment", "social_post"],
    verbose=True
)

# We can now run the chain with our randomized sentiment, and the social post!
jekyllhyde_chain.run({"sentiment": random_sentiment, "social_post": social_post})



> Entering new SequentialChain chain...

> Finished chain.


'"** *******, ******* **** ********* ** *** ******** ******** ** *** ****** ******* *********. ****\'** *******, ***\'ll ********* ******* *** ***** ***** ***** **** *** **** *** **** ******* *** **** *** ******** ** ** ***. #**************** #*********"'

In [ ]:
# As you see, the comment above is censored due to its inclusion of negative sentiments.